# Preparing Data for Modeling

In [5]:
import pandas as pd
import numpy as np
import sys

sys.path.append("..")

from util.parsing import get_variables

import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

from util.cleaning import agg_race_cah, agg_hispanicity, filter_na_response, has_bachelors, get_race, get_race_ta17, is_hispanic_cah, get_env_type, live_w_both_parents, is_race

## Read Data from SQL

We selected relevant variables and stored the data in several SQL tables in ```sql_logic.ipynb```. Now we use a SQL query to join these tables so we can create our dataframe.

In [6]:
from os import environ
from sqlalchemy import create_engine

uri = "postgres+psycopg2://zhou@localhost:5432/psid"
engine = create_engine(uri, echo=False)

```sql
SELECT * from ind17 
LEFT JOIN child02 on child02.indid01=ind17.indid01 AND child02.famid01=ind17.famid01 
LEFT JOIN assess ON assess.indid01 = ind17.indid01 AND assess.famid01=ind17.famid01 
LEFT JOIN demog ON demog.indid01 = ind17.indid01 AND demog.famid01=ind17.famid01 
LEFT JOIN fam01 ON fam01.indid01 = ind17.indid01 AND demog.famid68=ind17.famid68
WHERE ind17.indid01<>0 AND ind17.cds_interview=1
```

In [7]:
sql_get_join_data = "SELECT * FROM ind17 \
LEFT JOIN child02 on child02.indid01 = ind17.indid01 AND child02.famid01 = ind17.famid01 \
LEFT JOIN assess ON assess.indid01 = ind17.indid01 AND assess.famid01 = ind17.famid01 \
LEFT JOIN demog ON demog.indid01 = ind17.indid01 AND demog.famid01 = ind17.famid01 \
LEFT JOIN fam01 ON fam01.famid01 = ind17.famid01 AND fam01.famid68 = ind17.famid68 \
LEFT JOIN ta17 ON ta17.famid17 = ind17.famid17 AND ta17.indid17 = ind17.indid17 \
LEFT JOIN pcg02 ON pcg02.famid01 = ind17.famid01 AND pcg02.indid01 = ind17.indid01 \
LEFT JOIN wlth01 ON wlth01.famid01 = ind17.famid01 \
WHERE ind17.indid01<>0 AND ind17.cds_interview=1"

prelim_data_df = pd.read_sql_query(sql_get_join_data, con=engine)

Remove duplicate columns from the joins.

In [8]:
prelim_data_df = prelim_data_df.loc[:,~prelim_data_df.columns.duplicated()]

## Features of Interest

We isolate various features of interest through perusing the codebooks for the PSID surveys: math and reading assessment percentile scores from 2002; wealth and income of the family in 2001; whether the child lived with both parents in 2001; the household's food security status in 2001; the child's race; and "urbanicity" variable indicating where the child's neighborhood/county fall on the [Beale Rural-Urban Continuum](https://www.ers.usda.gov/data-products/rural-urban-continuum-codes.aspx).

Our continuous variables are ```math_score```, ```reading_score```, ```food_security```, ```wealth```, and ```income```.

Our binary categorical variables are ```white_only``` , ```black```, ```asian```, and ```hispanic```. 

We have one nominal categorical variables with several categories: our urbanicity variable, which we denote ```environment type```.  Its categories are ```met_central```, ```met_fringe```, ```met_small```, ```urb_met```, ```urb_nonmet```, and ```rural```.

Our response variable is ```grad_bach```, whether the individual received a bachelor's degree by 2017. 

We have an additional variable ```survey_weight``` which we use to weight samples during training and validation.

## Parse and Clean the Data

### Remove Non-responses

We remove all non-responses for our response variable.

In [9]:
prelim_data_df = prelim_data_df[prelim_data_df.apply(filter_na_response, axis=1)]

### Inferring Race/Ethnicity

The race/ethnicity data are scattered far and wide among many tables, due to some surveys only interviewing a subset of the study members; some individuals responding to one survey but not another; some surveys where individuals responded on their own behalf and others where parents and caretakers responded on a child's behalf, etc. We take pains to sift through this data.

We represent "White," "Black," "Asian," and "Hispanic" as binary 0-1 variables, with 1 indicating the individual is of that race and 0 indicating otherwise. Additionally, we have a "White Only" feature inferred from the aforementioned 4 variables; we use this feature along with "Black," "Asian," and "Hispanic" in our modeling.

In the Childhood and Adoption History data, parents and caretakers indicated the race of their charges. There are potentially multiple responses for each child, so we aggregate them in an array and merge with our dataframe.

In [10]:
get_cah = "SELECT * FROM cah"
cah_data_df = pd.read_sql_query(get_cah, con=engine)

# aggregate responses on the child's behalf
# the race_code variables indicate whether the responder said the child
# was white, asian, or black. separately, a hispanicity variable indicated
# the specific hispanic background of a child. 
cah_data_df_grouped = cah_data_df.groupby(["famid68", "indid68"]).agg({
    "hispanicity": agg_hispanicity,
    "race_code_cah_1": agg_race_cah,
    "race_code_cah_2": agg_race_cah,
    "race_code_cah_3": agg_race_cah
}).reset_index()

In [11]:
data_df = prelim_data_df.merge(cah_data_df_grouped, on=["famid68", "indid68"], how="left")

In [12]:
# fill these variables with empty arrays for children who didn't have anyone respond on their behalf during the CAH survey
data_df["race_code_cah_1"] = data_df["race_code_cah_1"].apply(lambda x: x if not x else [])
data_df["race_code_cah_2"] = data_df["race_code_cah_2"].apply(lambda x: x if not x else [])
data_df["race_code_cah_3"] = data_df["race_code_cah_3"].apply(lambda x: x if not x else [])

# get race listed in the child02 survey
data_df["race"] = data_df["race_code"].apply(get_race)

# get race listed in the ta17 survey
data_df["race17_1"] = data_df["race_code17_1"].apply(get_race_ta17)
data_df["race17_2"] = data_df["race_code17_2"].apply(get_race_ta17)

# infer race from all of the above
data_df["asian"] = data_df.apply(is_race("asian"), axis=1)
data_df["black"] = data_df.apply(is_race("black"), axis=1)
data_df["white"] = data_df.apply(is_race("white"), axis=1)
data_df["hispanic"] = data_df.apply(lambda row: 1 if is_race("hispanic")(row) or row["hispanicity"] else 0, axis=1)

data_df["white_only"] = data_df.apply(lambda row: 1 if row["white"] == 1 and row["hispanic"] == 0 and row["black"] == 0 and row["asian"] == 0 else 0, axis=1)

### Income and Wealth

We remove some extreme income/wealth outliers so as to improve our modeling. To do so, we cap income at 2 standard deviations above the mean.

In [13]:
# Handle income outliers by capping income and wealth at +2 std for now
inc_cap = data_df["total_fam_income00"].mean() + 2*data_df["total_fam_income00"].std()
data_df["total_fam_income00_cap"] = data_df["total_fam_income00"].clip(upper=inc_cap)
wealth_cap = data_df["wealth_w_equity01"].mean() + 2*data_df["wealth_w_equity01"].std()
data_df["wealth_w_equity01_cap"] = data_df["wealth_w_equity01"].clip(upper=wealth_cap)
data_df = data_df.replace({"math_score02": 999, "reading_score02": 999}, np.nan)

### Imputing Math and Reading Scores

We impute math and reading scores from the income and wealth variables.

N.B. We perform this imputation mainly as an exercise. We use wealth and income as they are the most convenient variables to impute with, and recognize that this strategy suffers from various flaws.

In [14]:
# Use income and wealth, our primary continuous variables, to impute math and reading scores
data_df = data_df.replace({"math_score02": 999, "reading_score02": 999}, np.nan)
temp_impute_df = data_df[["math_score02", "reading_score02", "wealth_w_equity01_cap", "total_fam_income00_cap"]]

imp = IterativeImputer(random_state=0, estimator=RandomForestRegressor())
data_df_imputed = pd.DataFrame(imp.fit_transform(temp_impute_df), index=temp_impute_df.index, columns=temp_impute_df.columns)

### Handle Other Variables

We construct our last few features and our response variable.

In [15]:
data_df["age"] = data_df["age_01"] + 16
data_df["live_w_both_parents"] = data_df.apply(live_w_both_parents, axis=1)
data_df["environment_type"] = data_df["rural_urban_code01"].apply(get_env_type)

data_df["grad_bach"] = data_df.apply(has_bachelors, axis=1).map({True: 1, False: 0})

### Last Cleaning Pass

We remove a few data points without and environment type, and limit our sample to people over age 22; younger individuals are increasingly unlikely to have graduated with a bachelor's degree.

In [16]:
data_df = data_df[~data_df["environment_type"].isnull()]
data_df = data_df[data_df["age"] >= 22]

Store the imputed data into our original dataframe.

In [17]:
data_df[["math_score02", "reading_score02", "wealth_w_equity01_cap", "total_fam_income00_cap"]] = data_df_imputed[["math_score02", "reading_score02", "wealth_w_equity01_cap", "total_fam_income00_cap"]]

## Final Touches

Rename variables for ease of use in modeling.

In [18]:
data_df = data_df.rename({"math_score02": "math_score", "reading_score02": "reading_score",\
                    "wealth_w_equity01_cap": "wealth", "total_fam_income00_cap": "income"}, axis=1)

Final list of features.

In [19]:
data_df = data_df[['math_score', 'reading_score', 'wealth', 'income', 'grad_bach', 'survey_weight', 'environment_type', 'age', 'white_only', 'black', 'asian', 'live_w_both_parents', 'food_security', 'hispanic']]

Write to a csv file.

In [23]:
data_df.to_csv("../data/data.csv", index=False)